In [0]:
from pyspark.sql import functions as F
bronze_path = "/Volumes/logistics/bronze/shipments_vol/"

all_files_df = (
    spark.createDataFrame(dbutils.fs.ls(bronze_path))
    .filter(F.col("name").endswith(".csv"))
    .select(F.col("name").alias("file_name"))
)
all_files_df.display()

file_name
shipment_daily=2023-01-02.csv
shipment_daily=2023-01-03.csv
shipment_daily=2023-01-04.csv
shipment_daily=2023-01-05.csv
shipment_daily=2023-01-07.csv
shipment_daily=2023-01-08.csv
shipment_daily=2023-01-09.csv
shipment_daily=2023-01-10.csv
shipment_daily=2023-01-11.csv
shipment_daily=2023-01-12.csv


In [0]:
processed_df = spark.table("logistics.control.processed_files")

new_files_df = all_files_df.join(
    processed_df,
    on="file_name",
    how="left_anti"
)
new_files_df.display()

file_name


In [0]:

from pyspark.sql import functions as F
from pyspark.sql.window import Window

new_files = [row.file_name for row in new_files_df.collect()]

for file_name in new_files:

    file_path = f"{bronze_path}/{file_name}"

    # Read CSV
    df = (
        spark.read
            .format("csv")
            .option("header", "true")
            .load(file_path)
    )

    # Ingest metadata
    df = (
        df
        .withColumn("ingest_ts", F.current_timestamp())
        .withColumn("ingest_date", F.current_date())
        .withColumn("source_file", F.lit(file_name))
    )

    # Normalize columns
    for c in df.columns:
        df = df.withColumnRenamed(c, c.lower().replace(" ", "_"))

    # Trim + case
    for c in ["carrier_id", "delivery_status"]:
        df = df.withColumn(c, F.upper(F.trim(F.col(c))))

    # Cast types
    df = (
        df
        .withColumn("shipment_cost", F.col("shipment_cost").cast("double"))
        .withColumn("shipment_date", F.to_date("shipment_date"))
        .withColumn("delivery_date", F.to_date("delivery_date"))
    )

    # Empty → null
    df = df.select([
        F.when(F.trim(F.col(c)) == "", None).otherwise(F.col(c)).alias(c)
        for c in df.columns
    ])

    # Dedup within file
    w = Window.partitionBy("shipment_id").orderBy(F.col("ingest_ts").desc())
    df = df.withColumn("rn", F.row_number().over(w)).filter("rn=1").drop("rn")

    # UNKNOWN carrier
    df = df.withColumn("carrier_id", F.coalesce(F.col("carrier_id"), F.lit("UNKNOWN")))

    # Cost rounding
    df = df.withColumn("shipment_cost", F.round(F.col("shipment_cost"), 2))

    # Outlier detection
    median_cost = df.approxQuantile("shipment_cost", [0.5], 0.01)[0]
    df = df.withColumn("dq_cost_outlier", F.col("shipment_cost") > median_cost * 10)

    # DQ flags
    df = (
        df
        .withColumn("dq_cost_null", F.col("shipment_cost").isNull())
        .withColumn("dq_cost_negative", F.col("shipment_cost") < 0)
        .withColumn("dq_cost_zero", F.col("shipment_cost") == 0)
        .withColumn("dq_date_anomaly", F.col("shipment_date") > F.col("delivery_date"))
        .withColumn(
            "dq_invalid_status",
            ~F.col("delivery_status").isin("DELIVERED","IN_TRANSIT","CANCELLED")
        )
    )

    # Score + severity
    df = df.withColumn(
        "dq_score",
        F.expr("""
            int(dq_cost_null) +
            int(dq_cost_negative) +
            int(dq_cost_zero) +
            int(dq_cost_outlier) +
            int(dq_date_anomaly) +
            int(dq_invalid_status)
        """)
    )

    df = df.withColumn(
        "dq_severity",
        F.when(F.col("dq_score") >= 2, "CRITICAL")
             .when(F.col("dq_score") == 1, "MAJOR")
             .otherwise("MINOR")
    )

    df = df.withColumn("is_quarantine", F.col("dq_severity") == "CRITICAL")

    # Append to Silver
    df.write \
      .format("delta") \
      .mode("append") \
      .saveAsTable("logistics.silver.shipments")

    # Mark file as processed
    spark.createDataFrame(
        [(file_name,)],
        ["file_name"]
    ).withColumn(
        "processed_ts", F.current_timestamp()
    ).write.mode("append").saveAsTable("logistics.control.processed_files")


In [0]:
%sql
SELECT COUNT(*) AS total_records
FROM logistics.silver.shipments;

total_records
300984


In [0]:
%sql
SELECT 
    COUNT(DISTINCT shipment_id) AS distinct_shipments
FROM logistics.silver.shipments;


distinct_shipments
258955


In [0]:
%sql
select * FROM logistics.control.processed_files

file_name,processed_ts
shipment_daily=2023-01-12.csv,2026-02-09T13:20:32.853Z
shipment_daily=2023-01-08.csv,2026-02-09T12:52:22.459Z
shipment_daily=2023-01-11.csv,2026-02-09T13:20:28.519Z
shipment_daily=2023-01-10.csv,2026-02-09T13:20:23.733Z
shipment_daily=2023-01-07.csv,2026-02-09T12:52:18.015Z
shipment_daily=2023-01-13.csv,2026-02-09T15:11:35.463Z
shipment_daily=2023-01-09.csv,2026-02-09T12:52:27.307Z
shipment_daily=2023-01-14.csv,2026-02-09T15:11:40.615Z
shipment_daily=2023-01-05.csv,2026-02-09T12:52:12.635Z
shipment_daily=2023-01-15.csv,2026-02-09T15:16:33.653Z


In [0]:
%sql
SELECT 
    source_file,
    COUNT(*) AS records_loaded
FROM logistics.silver.shipments
GROUP BY source_file
ORDER BY source_file;


source_file,records_loaded
shipment_daily=2023-01-02.csv,84058
shipment_daily=2023-01-03.csv,41838
shipment_daily=2023-01-04.csv,41798
shipment_daily=2023-01-05.csv,41871
shipment_daily=2023-01-07.csv,42087
shipment_daily=2023-01-08.csv,6155
shipment_daily=2023-01-09.csv,6211
shipment_daily=2023-01-10.csv,6164
shipment_daily=2023-01-11.csv,6169
shipment_daily=2023-01-12.csv,6163


In [0]:
%sql
SELECT COUNT(*) AS files_processed
FROM logistics.control.processed_files;


files_processed
13


In [0]:
%sql
SELECT COUNT(*) AS null_cost_records
FROM logistics.silver.shipments
WHERE dq_cost_null = true;


null_cost_records
11887


In [0]:
%sql
SELECT COUNT(*) AS date_anomaly_records
FROM logistics.silver.shipments
WHERE dq_date_anomaly = true;


date_anomaly_records
0


In [0]:
%sql
SELECT shipment_id, shipment_date, delivery_date
FROM logistics.silver.shipments
WHERE dq_date_anomaly = true
LIMIT 20;


shipment_id,shipment_date,delivery_date


In [0]:
%sql
SELECT 
    dq_severity,
    COUNT(*) AS record_count
FROM logistics.silver.shipments
GROUP BY dq_severity
ORDER BY dq_severity;


dq_severity,record_count
MAJOR,4246
MINOR,296738


In [0]:
%sql
SELECT
    COUNT(*) AS total_records,
    SUM(CASE WHEN dq_severity = 'CRITICAL' THEN 1 ELSE 0 END) AS critical_records,
    SUM(CASE WHEN dq_severity = 'MAJOR' THEN 1 ELSE 0 END) AS major_records,
    SUM(CASE WHEN dq_severity = 'MINOR' THEN 1 ELSE 0 END) AS minor_records
FROM logistics.silver.shipments;


total_records,critical_records,major_records,minor_records
300984,0,4246,296738


In [0]:
from pyspark.sql.functions import col
shipments_df = spark.table("logistics.silver.shipments")
silver_cleaned_df = shipments_df.filter(col("dq_severity") == "MINOR")
quarantine_df = shipments_df.filter(
    col("dq_severity").isin("MAJOR", "CRITICAL")
)


In [0]:

silver_cleaned_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("logistics.silver.silver_cleaned")
quarantine_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("logistics.silver.quarantine")


In [0]:
%sql
SELECT COUNT(*) FROM logistics.silver.silver_cleaned;
SELECT COUNT(*) FROM logistics.silver.quarantine;


COUNT(*)
22884


In [0]:
silver_cleaned_df = spark.table("logistics.silver.silver_cleaned")

silver_cleaned_df \
    .coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("abfss://silver@ltimc1sacc.dfs.core.windows.net/silver_cleaned_csv/")

quarantine_df = spark.table("logistics.silver.quarantine")

quarantine_df \
    .coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("abfss://silver@ltimc1sacc.dfs.core.windows.net/quarantine_csv/")


**Gold Layer**

dim_date

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.dim_date AS
SELECT
  CAST(date_format(shipment_date, 'yyyyMMdd') AS INT) AS date_sk,
  shipment_date AS full_date,
  year(shipment_date) AS year,
  month(shipment_date) AS month,
  day(shipment_date) AS day
FROM (
  SELECT DISTINCT shipment_date
  FROM logistics.silver.silver_cleaned
)
ORDER BY shipment_date;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM logistics.gold.dim_date

date_sk,full_date,year,month,day
20230102,2023-01-02,2023,1,2
20230103,2023-01-03,2023,1,3
20230104,2023-01-04,2023,1,4
20230105,2023-01-05,2023,1,5
20230107,2023-01-07,2023,1,7
20230108,2023-01-08,2023,1,8
20230109,2023-01-09,2023,1,9
20230110,2023-01-10,2023,1,10
20230111,2023-01-11,2023,1,11
20230112,2023-01-12,2023,1,12


In [0]:
regions_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/Volumes/logistics/gold/dimtables/regions.csv")

regions_df.createOrReplaceTempView("regions_stg")

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.dim_region AS
SELECT
  row_number() OVER (ORDER BY region_id) AS region_sk,
  region_id,
  region_name
FROM regions_stg;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM logistics.gold.dim_region

region_sk,region_id,region_name
1,R01,North
2,R02,South
3,R03,East
4,R04,West
5,R05,Central
6,R06,Northeast
7,R07,Northwest
8,R08,Southeast


In [0]:
warehouses_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/Volumes/logistics/gold/dimtables/warehouses.csv")

warehouses_df.createOrReplaceTempView("warehouses_stg")

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.dim_warehouse AS
SELECT
  row_number() OVER (ORDER BY warehouse_id) AS warehouse_sk,
  warehouse_id,
  warehouse_name,
  city,
  capacity_tpd
FROM warehouses_stg;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM logistics.gold.dim_warehouse

warehouse_sk,warehouse_id,warehouse_name,city,capacity_tpd
1,W01,Warehouse-W01,Ahmedabad,246
2,W02,Warehouse-W02,Delhi,259
3,W03,Warehouse-W03,Lucknow,270
4,W04,Warehouse-W04,Hyderabad,88
5,W05,Warehouse-W05,Chennai,104
6,W06,Warehouse-W06,Kolkata,296
7,W07,Warehouse-W07,Bengaluru,77
8,W08,Warehouse-W08,Jaipur,298
9,W09,Warehouse-W09,Ahmedabad,260
10,W10,Warehouse-W10,Kolkata,317


In [0]:
carriers_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/Volumes/logistics/gold/dimtables/carriers.csv")

carriers_df.createOrReplaceTempView("carriers_stg")


In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.dim_carrier AS
SELECT
  row_number() OVER (ORDER BY carrier_id) AS carrier_sk,
  carrier_id,
  carrier_name,
  mode
FROM carriers_stg;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM logistics.gold.dim_carrier

carrier_sk,carrier_id,carrier_name,mode
1,C01,BlueDart,Air
2,C02,Delhivery,Road
3,C03,DTDC,Road
4,C04,EcomExpress,Road
5,C05,FedEx,Air
6,C06,DHL,Air
7,C07,IndiaPost,Rail
8,C08,XpressBees,Road
9,C09,ShadowCarrier,Road
10,C10,QuickShip,Road


In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.fact_shipments AS
SELECT
    s.shipment_id,

    COALESCE(dc.carrier_sk, -1) AS carrier_sk,
    dw.warehouse_sk,
    dr.region_sk,
    dd.date_sk,

    s.shipment_cost,
    s.delivery_days,
    s.priority_level,
    s.is_fragile,
    s.delivery_status,
    s.payment_type

FROM logistics.silver.silver_cleaned s

LEFT JOIN logistics.gold.dim_carrier dc
    ON s.carrier_id = dc.carrier_id

LEFT JOIN logistics.gold.dim_warehouse dw
    ON s.warehouse_id = dw.warehouse_id

LEFT JOIN logistics.gold.dim_region dr
    ON s.region_id = dr.region_id

LEFT JOIN logistics.gold.dim_date dd
    ON s.shipment_date = dd.full_date

WHERE s.shipment_cost IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM logistics.gold.fact_shipments

shipment_id,carrier_sk,warehouse_sk,region_sk,date_sk,shipment_cost,delivery_days,priority_level,is_fragile,delivery_status,payment_type
SHP20230101003485,-1,6,8,20230102,1272.31,3,Medium,False,DELIVERED,Prepaid
SHP20230101023925,6,6,8,20230102,1640.63,6,Medium,False,DELIVERED,Prepaid
SHP20230101031025,6,2,4,20230102,837.53,3.0,Low,False,DELIVERED,Prepaid
SHP20230101032585,1,2,4,20230102,1491.5,3,High,True,DELIVERED,Prepaid
SHP20230101037695,11,16,2,20230102,1065.09,1,Low,False,DELIVERED,Prepaid
SHP20230101041245,11,6,8,20230102,1414.42,4.0,Low,False,DELIVERED,COD
SHP20230101054020,6,21,7,20230102,692.55,7.0,Low,False,DELIVERED,COD
SHP20230101059695,1,8,5,20230102,799.32,5,Low,False,DELIVERED,COD
SHP20230101067360,1,1,1,20230102,1962.27,7,Medium,True,DELIVERED,COD
SHP20230101069350,6,31,7,20230102,788.89,6.0,Low,False,DELIVERED,Prepaid


In [0]:
dimdate_df = spark.table("logistics.gold.dim_date")
dimdate_df.coalesce(1).write \
  .mode("overwrite") \
  .option("header", "true") \
  .csv("abfss://gold@ltimc1sacc.dfs.core.windows.net/dimdate/")

dimcarrier_df = spark.table("logistics.gold.dim_carrier")
dimcarrier_df.coalesce(1).write \
  .mode("overwrite") \
  .option("header", "true") \
  .csv("abfss://gold@ltimc1sacc.dfs.core.windows.net/dimcarrier/")

dimregion_df = spark.table("logistics.gold.dim_region")
dimregion_df.coalesce(1).write \
  .mode("overwrite") \
  .option("header", "true") \
  .csv("abfss://gold@ltimc1sacc.dfs.core.windows.net/dimregion/")

dimwarehouse_df = spark.table("logistics.gold.dim_warehouse")
dimwarehouse_df.coalesce(1).write \
  .mode("overwrite") \
  .option("header", "true") \
  .csv("abfss://gold@ltimc1sacc.dfs.core.windows.net/dimwarehouse/")

fact_df = spark.table("logistics.gold.fact_shipments")
fact_df.coalesce(1).write \
  .mode("overwrite") \
  .option("header", "true") \
  .csv("abfss://gold@ltimc1sacc.dfs.core.windows.net/factshipments/")



In [0]:
%sql
SELECT
    COUNT(*) AS total_rows,
    SUM(CASE WHEN carrier_sk IS NULL THEN 1 ELSE 0 END) AS null_carrier,
    SUM(CASE WHEN warehouse_sk IS NULL THEN 1 ELSE 0 END) AS null_warehouse,
    SUM(CASE WHEN region_sk IS NULL THEN 1 ELSE 0 END) AS null_region
FROM logistics.gold.fact_shipments;


total_rows,null_carrier,null_warehouse,null_region
1524568,0,0,0


Daily Cost Trend Analysis

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_daily_cost AS
SELECT f.date_sk, full_date, year, month,
       SUM(shipment_cost) as total_cost,
       AVG(shipment_cost) as avg_cost,
       COUNT(*) as shipment_count
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_date d ON f.date_sk = d.date_sk
WHERE shipment_cost IS NOT NULL
GROUP BY f.date_sk, full_date, year, month
ORDER BY full_date;

num_affected_rows,num_inserted_rows


Regional Cost Variance Analysis

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_region_cost AS
SELECT r.region_sk, r.region_name,
       SUM(shipment_cost) as total_cost,
       AVG(shipment_cost) as avg_cost,
       COUNT(*) as shipment_count,
       COUNT(DISTINCT f.carrier_sk) as carrier_count
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_region r ON f.region_sk = r.region_sk
GROUP BY r.region_sk, r.region_name;


num_affected_rows,num_inserted_rows


Warehouse Capacity Stress Index

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_warehouse_stress AS
SELECT w.warehouse_sk, w.warehouse_name, w.city, w.capacity_tpd,
       COUNT(*) as shipment_count,
       COUNT(*) * 1.0 / NULLIF(w.capacity_tpd, 0) as stress_index
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_warehouse w ON f.warehouse_sk = w.warehouse_sk
GROUP BY w.warehouse_sk, w.warehouse_name, w.city, w.capacity_tpd
ORDER BY stress_index DESC;


num_affected_rows,num_inserted_rows


Carrier Performance Analytics

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_carrier_perf AS
SELECT c.carrier_sk, c.carrier_name, c.mode,
       COUNT(*) as total_shipments,
       SUM(shipment_cost) as total_cost,
       AVG(CAST(TRY_CAST(delivery_days AS FLOAT) AS FLOAT)) as avg_days,
       AVG(CASE WHEN TRY_CAST(delivery_days AS FLOAT) <= 3.0 THEN 1.0 ELSE 0.0 END) as ontime_pct,
       COUNT(CASE WHEN TRY_CAST(delivery_days AS DOUBLE) > 7.0 THEN 1 END) as late_shipments
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_carrier c ON f.carrier_sk = c.carrier_sk
GROUP BY c.carrier_sk, c.carrier_name, c.mode;


num_affected_rows,num_inserted_rows


Payment Type Performance Analysis

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_payment_perf AS
SELECT payment_type,
       COUNT(*) as shipment_count,
       AVG(shipment_cost) as avg_cost,
       AVG(TRY_CAST(delivery_days AS DOUBLE)) as avg_days,
       SUM(shipment_cost) as total_cost
FROM logistics.gold.fact_shipments
GROUP BY payment_type;


num_affected_rows,num_inserted_rows


Priority Service Profitability

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_priority_status AS
SELECT priority_level, delivery_status,
       COUNT(*) as shipment_count,
       AVG(shipment_cost) as avg_cost,
       AVG(TRY_CAST(delivery_days AS DOUBLE)) as avg_days
FROM logistics.gold.fact_shipments
GROUP BY priority_level, delivery_status;


num_affected_rows,num_inserted_rows


Fragile Item Cost Premium Analysis

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_fragile_cost AS
SELECT is_fragile,
       COUNT(*) as shipment_count,
       AVG(shipment_cost) as avg_cost,
       SUM(shipment_cost) as total_cost,
       AVG(TRY_CAST(delivery_days AS DOUBLE)) as avg_days
FROM logistics.gold.fact_shipments
GROUP BY is_fragile;


num_affected_rows,num_inserted_rows


Daily Data Quality Monitoring

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_dq_daily AS
SELECT d.date_sk, d.full_date,
       COUNT(*) as total_records,
       SUM(CASE WHEN shipment_cost IS NULL THEN 1 ELSE 0 END) as null_costs,
       SUM(CASE WHEN TRY_CAST(delivery_days AS DOUBLE) IS NULL THEN 1 ELSE 0 END) as null_days,
       AVG(CASE WHEN shipment_cost IS NOT NULL THEN 1.0 ELSE 0 END) as dq_score
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_date d ON f.date_sk = d.date_sk
GROUP BY d.date_sk, d.full_date;


num_affected_rows,num_inserted_rows


Carrier Efficiency Frontier

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_carrier_efficiency AS
SELECT c.carrier_name, c.mode,
       AVG(shipment_cost) as avg_cost,
       AVG(CASE WHEN TRY_CAST(delivery_days AS DOUBLE) <= 3.0 THEN 1.0 ELSE 0 END) as ontime_pct,
       COUNT(*) as volume,
       AVG(TRY_CAST(delivery_days AS DOUBLE)) as avg_delay,
       COUNT(CASE WHEN TRY_CAST(delivery_days AS DOUBLE) > 7.0 THEN 1 END)*1.0/COUNT(*) as late_pct
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_carrier c ON f.carrier_sk = c.carrier_sk
GROUP BY c.carrier_name, c.mode;


num_affected_rows,num_inserted_rows


SLA Breach Risk Quadrant

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_sla_breach AS
SELECT c.carrier_name,
       COUNT(CASE WHEN TRY_CAST(delivery_days AS DOUBLE) <= 3.0 THEN 1 END) as ontime,
       COUNT(CASE WHEN TRY_CAST(delivery_days AS DOUBLE) > 3.0 THEN 1 END) as breached,
       COUNT(CASE WHEN TRY_CAST(delivery_days AS DOUBLE) > 3.0 THEN 1 END)*1.0/COUNT(*) as breach_pct,
       COUNT(*) as total
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_carrier c ON f.carrier_sk = c.carrier_sk
GROUP BY c.carrier_name;


num_affected_rows,num_inserted_rows


Monthly Executive KPI Tracking

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_monthly_kpi AS
SELECT d.year, d.month, d.full_date,
       COUNT(*) as shipments,
       SUM(shipment_cost) as total_cost,
       AVG(shipment_cost) as avg_cost,
       AVG(TRY_CAST(delivery_days AS DOUBLE)) as avg_days,
       AVG(CASE WHEN TRY_CAST(delivery_days AS DOUBLE) <= 3.0 THEN 1.0 ELSE 0 END) as ontime_pct
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_date d ON f.date_sk = d.date_sk
GROUP BY d.year, d.month, d.full_date;


num_affected_rows,num_inserted_rows


Delivery Success Rate Analytics

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_delivery_success AS
SELECT d.date_sk, d.full_date, d.year, d.month,
       COUNT(*) as total_shipments,
       SUM(CASE WHEN TRY_CAST(f.delivery_days AS DOUBLE) <= 3 THEN 1 ELSE 0 END) as success_deliveries,
       SUM(CASE WHEN TRY_CAST(f.delivery_days AS DOUBLE) <= 3 THEN 1 ELSE 0 END)*100.0/COUNT(*) as success_pct,
       AVG(TRY_CAST(f.delivery_days AS DOUBLE)) as avg_days
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_date d ON f.date_sk = d.date_sk
GROUP BY d.date_sk, d.full_date, d.year, d.month;


num_affected_rows,num_inserted_rows


Fragile Damage Proxy Analytics

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_fragile_damage AS
SELECT d.date_sk, d.full_date, d.year, d.month,
       COUNT(*) FILTER (WHERE f.is_fragile = true) as fragile_shipments,
       COUNT(*) FILTER (WHERE f.is_fragile = true AND TRY_CAST(f.delivery_days AS DOUBLE) > 5) as damage_proxy,
       COUNT(*) FILTER (WHERE f.is_fragile = true AND TRY_CAST(f.delivery_days AS DOUBLE) > 5)*100.0 / 
       NULLIF(COUNT(*) FILTER (WHERE f.is_fragile = true), 0) as damage_pct,
       AVG(TRY_CAST(f.delivery_days AS DOUBLE)) FILTER (WHERE f.is_fragile = true) as fragile_avg_days
FROM logistics.gold.fact_shipments f
JOIN logistics.gold.dim_date d ON f.date_sk = d.date_sk
GROUP BY d.date_sk, d.full_date, d.year, d.month;


num_affected_rows,num_inserted_rows


Data Quality Matrix Dashboard

In [0]:
%sql
CREATE OR REPLACE TABLE logistics.gold.agg_dq_matrix AS
SELECT s.ingest_date,
       COUNT(*) as total_records,
       SUM(CASE WHEN s.dq_cost_null = true THEN 1 ELSE 0 END) as null_cost_count,
       SUM(CASE WHEN s.dq_cost_null = true THEN 1 ELSE 0 END)*100.0/COUNT(*) as null_cost_pct,
       SUM(CASE WHEN s.dq_cost_outlier = true THEN 1 ELSE 0 END) as outlier_count,
       SUM(CASE WHEN s.dq_cost_outlier = true THEN 1 ELSE 0 END)*100.0/COUNT(*) as outlier_pct,
       SUM(CASE WHEN s.dq_date_anomaly = true THEN 1 ELSE 0 END) as anomaly_count,
       SUM(CASE WHEN s.dq_date_anomaly = true THEN 1 ELSE 0 END)*100.0/COUNT(*) as anomaly_pct,
       SUM(CASE WHEN s.dq_score >= 2 THEN 1 ELSE 0 END) as critical_count
FROM logistics.silver.shipments s
GROUP BY s.ingest_date
ORDER BY s.ingest_date;


num_affected_rows,num_inserted_rows
